In [1]:
### Un'applicazione che mescola in modo interessante Python e un database 
### per calcolare il percorso tra due stazioni nella metro di Parigi
### attivare Uwamp: Apache web server ed interprete SQL
### QUESTA E' LA VERSIONE PUBBLICATA su github e presentata sul prossimo video di youtube
### LA VERSIONE ORIGINALE E' IN "c:/.../Dropbox/Scuola/Python/corso Python/Lezione33.ipynb"

In [2]:
### il file "c:\\documenti\\demografia\\Paris\\metro.csv" è ottenuto
### con il copia-incolla dalla pagina web
### "https://en.wikipedia.org/wiki/List_of_Paris_M%C3%A9tro_stations"
### con qualche piccola modifica mediante un foglio elettronico

import pandas as pd
import numpy as np
#url = "c:\\documenti\\demografia\\Paris\\metro.csv"
url = "metro.csv"
df = pd.read_csv(url, header = None)
a = np.array(df)
b = []
n = 0
for i in range(len(a)):
    c = a[i][0].split(sep = ";")
    if (c[0] != ''):
        if ('(' in c[0] and i > 0):
            b[n-1][0] = b[n-1][0] + " " + c[0]
        else:
            n = n + 1
            b.append(c)
for i in range(len(b)):
    d = b[i][1].replace("Paris Métro Line","").strip()
    d = d.split(sep = " ")
    b[i][1] = d
### creazione della lista delle linee
c = []
for i in range(len(b)):
    for j in range(len(b[i][1])):
        if (b[i][1][j] not in c):
            c.append(b[i][1][j])
### creazione della lista delle stazioni di ogni linea
d = []
for i in range(len(c)):
    e = []
    for j in range(len(b)):
        if (c[i] in b[j][1]):
            e.append(j)
    d.append(e)


In [3]:
### a questo punto si deve lanciare Uwamp ed autenticarsi su PHPmyadmin 
### con user="root" e password="root" (da usare solo in locale)
### creazione e popolazione database ‘Paris_metro’ (tabella “stazioni”) ### 
import pandas as pd
import numpy as np
import pymysql
conn = pymysql.connect(host='localhost', user='root', password='root')
cur = conn.cursor()
query = 'DROP DATABASE IF EXISTS Paris_metro'
n_rows = cur.execute(query)
query = 'CREATE DATABASE Paris_metro'
n_rows = cur.execute(query)
query = 'DROP TABLE IF EXISTS Paris_metro.stazioni'
n_rows = cur.execute(query)
query = '''
CREATE TABLE Paris_metro.stazioni ( 
    IDstazione INT NOT NULL AUTO_INCREMENT, 
    nome TEXT NOT NULL, 
    PRIMARY KEY (IDstazione));
'''
n_rows = cur.execute(query)
for i in range(len(b)):
    query = "INSERT INTO Paris_metro.stazioni(IDstazione,nome) "
    query = query + "VALUES(NULL,\"" + b[i][0] +"\");"
    n_rows = cur.execute(query)
    conn.commit()


In [4]:
### creazione e popolazione database ‘Paris_metro’ (tabella “linee”) ### 
import pandas as pd
import numpy as np
import pymysql
conn = pymysql.connect(host='localhost', user='root', password='root')
cur = conn.cursor()
#### query = 'DROP DATABASE IF EXISTS Paris_metro'
#### n_rows = cur.execute(query)
#### query = 'CREATE DATABASE Paris_metro'
#### n_rows = cur.execute(query)
query = 'DROP TABLE IF EXISTS Paris_metro.linee'
n_rows = cur.execute(query)
query = '''
CREATE TABLE Paris_metro.linee ( 
    IDlinea INT NOT NULL AUTO_INCREMENT, 
    nome TEXT NOT NULL, 
    PRIMARY KEY (IDlinea));
'''
n_rows = cur.execute(query)
for i in range(len(c)):
    query = "INSERT INTO Paris_metro.linee(IDlinea,nome) "
    query = query + "VALUES(NULL,\"" + str(c[i]) +"\");"
    n_rows = cur.execute(query)
    conn.commit()

In [5]:
### creazione e popolazione database ‘Paris_metro’ (tabella “linee_stazioni”) ### 
import pandas as pd
import numpy as np
import pymysql
conn = pymysql.connect(host='localhost', user='root', password='root')
cur = conn.cursor()
#### query = 'DROP DATABASE IF EXISTS Paris_metro'
#### n_rows = cur.execute(query)
#### query = 'CREATE DATABASE Paris_metro'
#### n_rows = cur.execute(query)
query = 'DROP TABLE IF EXISTS Paris_metro.linee_stazioni'
n_rows = cur.execute(query)
query = '''
CREATE TABLE Paris_metro.linee_stazioni ( 
    IDlinea_stazione INT NOT NULL AUTO_INCREMENT, 
    CODlinea INT NOT NULL,
    CODstazione INT NOT NULL,
    posizione1 INT,
    posizione2 INT, 
    PRIMARY KEY (IDlinea_stazione));
'''
n_rows = cur.execute(query)
for i in range(len(d)):
    for j in range(len(d[i])):
        query = "INSERT INTO Paris_metro.linee_stazioni(IDlinea_stazione,CODlinea,CODstazione,posizione1,posizione2) "
        query = query + "VALUES(NULL," + str(i+1) + "," + str(d[i][j]+1) + ",NULL,NULL);" 
### i + 1 e d[i][j]+1 perché nel database 
### la numerazione delle stazioni (“IDstazione” nella tabella “stazioni” )
### e delle linee ("IDlinea" nella tabella "linee")
### parte da 1 e non da 0
        n_rows = cur.execute(query)
        conn.commit()


In [6]:
### eseguire la query sul database come test
### nel caso che le stazioni di partenza e di arrivo siano 
### sulla stessa linea
### query per stazioni sulla stessa linea
### stampa la lista di tutte le stazioni raggiungibili da una stazione
### di partenza (senza cambi di linea)
'''
SELECT `linee`.`nome`,`stazioni2`.`nome` 
FROM `linee`,
    `linee_stazioni` AS `linee_stazioni1`,
    `linee_stazioni` AS `linee_stazioni2`,
    `stazioni` AS `stazioni1`,
    `stazioni` AS `stazioni2`
WHERE `stazioni1`.`nome` = 'Republique'
AND `stazioni1`.`IDstazione` = `linee_stazioni1`.`CODstazione`
AND `linee_stazioni1`.`CODlinea` = `linee`.`IDlinea`
AND `linee`.`IDlinea` = `linee_stazioni2`.`CODlinea`
AND `linee_stazioni2`.`CODstazione` = `stazioni2`.`IDstazione`
'''

"\nSELECT `linee`.`nome`,`stazioni2`.`nome` \nFROM `linee`,\n    `linee_stazioni` AS `linee_stazioni1`,\n    `linee_stazioni` AS `linee_stazioni2`,\n    `stazioni` AS `stazioni1`,\n    `stazioni` AS `stazioni2`\nWHERE `stazioni1`.`nome` = 'Republique'\nAND `stazioni1`.`IDstazione` = `linee_stazioni1`.`CODstazione`\nAND `linee_stazioni1`.`CODlinea` = `linee`.`IDlinea`\nAND `linee`.`IDlinea` = `linee_stazioni2`.`CODlinea`\nAND `linee_stazioni2`.`CODstazione` = `stazioni2`.`IDstazione`\n"

In [7]:
### la query precedente trasformata in funzione Python
### SI PUO' RIPARTIRE DA QUI SE IL DATABASE E' GIA' PRONTO
def destinazioni(partenza):
    import pandas as pd
    import numpy as np
    import pymysql
    conn = pymysql.connect(host='localhost', user='root', password='root', database='paris_metro')
    cur = conn.cursor()
    query = '''
    SELECT `linee`.`nome`,`stazioni2`.`nome` 
    FROM `linee`,
        `linee_stazioni` AS `linee_stazioni1`,
        `linee_stazioni` AS `linee_stazioni2`,
        `stazioni` AS `stazioni1`,
        `stazioni` AS `stazioni2`'''
    partenza = "WHERE `stazioni1`.`nome` = \"" + partenza
    clause = '''\" AND `stazioni1`.`IDstazione` = `linee_stazioni1`.`CODstazione`
        AND `linee_stazioni1`.`CODlinea` = `linee`.`IDlinea`
        AND `linee`.`IDlinea` = `linee_stazioni2`.`CODlinea`
        AND `linee_stazioni2`.`CODstazione` = `stazioni2`.`IDstazione`
    '''
    query = (query + partenza + clause).strip('\n')
###    print(query)
    n_rows = cur.execute(query)
    r = []
    for i in range(n_rows):
        string = cur.fetchone()
        r.append(string)
###        print(string)
###        print(r[i][1])   
    return r

In [8]:
### percorso tra due stazioni ()
partenza = "Gambetta"
arrivo = "Rambuteau"
elenco = destinazioni(partenza) ### stazioni sulla stessa linea
for i in range(len(elenco)):
    if (arrivo == elenco[i][1]):
        print(elenco[i][0])
        break
    elencobis = destinazioni(elenco[i][1]) ### stazioni su linee differenti (un cambio)
    for j in range(len(elencobis)):
        if (arrivo == elencobis[j][1]):
            print(str(elenco[i])+" * " + str(elencobis[j]))
            break

In [9]:
### il seguente programma in Python usa la query precedente in una funzione
### chiamata ricorsivamente per esplorare la rete (grafo) della metropolitana
### 
### NOTA BENE: 
### 1) l'uso della query SQL ci evita di scrivere l'algoritmo per esplorare il grafo
### 2) l'uso del Python ci permette di automatizzare l'utilizzo della query e quindi di usarla 
### ricorsivamente

def percorso(partenza,arrivo,lista):
    import random as rd
    elenco = destinazioni(partenza)
    for i in range(len(elenco)):
        if (arrivo == elenco[i][1]):
            print(elenco[i][0] + " " + elenco[i][1])
            lista.append(elenco[i])
###            print("*")
            return lista
    for i in range(len(elenco)):
        elencobis = destinazioni(elenco[i][1])
        for j in range(len(elencobis)):
            if (arrivo == elencobis[j][1]):
                print(elenco[i][0] + " " + elenco[i][1])
                lista.append(elencobis[j])
                lista.append(elenco[i])
###                print("**")
                return lista
    for i in range(len(elenco)):   ### da controllare nel caso di 2 o più cambi
        elencobis = destinazioni(elenco[i][1])
        for j in range(len(elencobis)):
            m = len(elencobis)
            k = (j + int(m * rd.random())) % m
            q = len(lista)
            lista = percorso(elencobis[k][1],arrivo,lista)
            if (len(lista) > q and lista[-1] != [] and elencobis[k] not in lista 
                and lista[-1][1] not in elencobis[:][1]):
###                print(elencobis[k])
                if (elencobis[k][0] != lista[-1][0]):
                    lista.append(elencobis[k])
###                print("***")
                return lista
    return lista

In [10]:
### controllare gli accenti (l'elenco dei nomi corretti delle stazioni è nella tabella 'stazioni' del database)
### con "Republique" al posto di "République" la procedura non funziona
### a = percorso("Gambetta","Rambuteau (Centre Georges Pompidou)",[]) ### percorso diretto
a = percorso("Gambetta","Voltaire (Léon Blum)",[]) ### percorso diretto
### a = percorso("Ourcq","Bastille",[]) ### percorso diretto
a.reverse()
print(a)

3 Havre - Caumartin
[('3', 'Havre - Caumartin'), ('9', 'Voltaire (Léon Blum)')]


In [11]:
#a = percorso("Gambetta","Victor Hugo",[])  ### percorso con 1 cambio
a = percorso("Gambetta","Rome",[])
a.reverse()
print(a)

3 Père Lachaise
[('3', 'Père Lachaise'), ('2', 'Rome')]


In [12]:
#### a = percorso("Gambetta","Cambronne",[]) ### percorso con 2 o più cambi
### bisogna selezionare due linee che non si incrociano (come la 3 e la 6)
a = percorso("Argentine","Porte de Pantin (Parc de la Villette)",[])
a = percorso("Blanche","Cardinal Lemoine",[])
a = percorso("Gallieni (Parc de Bagnolet)","Cambronne",[])
a.reverse()
print(a)

1 Bastille
7 Jussieu
9 Nation (9: Place des Antilles)
[('3', 'Havre - Caumartin'), ('9', 'Nation (9: Place des Antilles)'), ('6', 'Cambronne')]


In [13]:
### a = percorso("Bolivar","Passy",[]) ### percorso libero
a = percorso("Ourcq","Lourmel",[])
a = percorso("Châtelet (7: Pont au Change)","Bir-Hakeim (Tour Eiffel)",[])
a = percorso("Gallieni (Parc de Bagnolet)","Bir-Hakeim (Tour Eiffel)",[])
a = percorso("Charles Michels","Anvers (Sacré-Cœur)",[])
a = percorso("Bastille","Vaneau",[])
a = percorso("Les Gobelins","Lourmel",[])
a = percorso("Kléber","Porte de Bagnolet",[])
a.reverse()
print(a)

5 Bastille
4 Denfert-Rochereau (Colonel Rol-Tanguy)
2 Charles de Gaulle - Étoile
9 Nation (9: Place des Antilles)
8 La Motte-Picquet - Grenelle
7 Opéra
8 Opéra
[('6', 'La Motte-Picquet - Grenelle'), ('8', 'Opéra'), ('3', 'Porte de Bagnolet')]


In [14]:
### legge I file delle linee per caricare l’ordine delle stazioni e popolare la tabella “linee_stazioni” (colonne
### “posizione1” e “posizione2”)
### i file sono stati ricavati con il copia-incolla dalla pagina web: 
### https://www.pariscityvision.com/en/paris/metro-line-1 e seguenti

In [15]:
import pandas as pd
l = []
### m = ['1','2','3','4','5','6', '7','7a','7b','8','9', '10','11','12','13','13a','13b','14']
m = ['12','4','2','9','3','1','11','7','10','8','13','5','6','14']  ### dati ordinati con la tabella ‘linee’ del db
for i in range(len(m)):
    filename = "c:\\documenti\\demografia\\Paris\\linee\\Ligne" + m[i] + ".csv"
    df = pd.read_csv(filename, header=None, sep='\t', encoding='utf-8')
##    df = pd.read_csv(filename, header=None, encoding='utf-8')
    dl = df.iloc[1:].values
    l.append(dl)
import numpy as np
import pymysql
conn = pymysql.connect(host='localhost', user='root', password='root', database='paris_metro')
cur = conn.cursor()
p = []
c = 0
for i in range(len(l)):
    n = len(l[i])
    q = []
    for j in range(n):
        nome = str(l[i][j])[2:-2]
        for k in range(len(nome)-2):
            if (nome[k:k+2] == " :"):
                nome = nome[:k]
            if (nome[-4:] == '\\xa0'):
                nome = nome[:-4]
            nome = nome.replace("’","'")
            nome = nome.replace("–","-")
        print(nome)
        query = "SELECT `IDstazione` FROM `stazioni` WHERE `stazioni`.`nome` LIKE \""
        query = query + nome[:-1] + "%\""
        n_rows = cur.execute(query)
        stringa = cur.fetchone()
        if (stringa == None):
            stringa = "-1"
        else:
            c = c + 1
            stringa = str(stringa)[1:-2]
##        print(stringa + " " + nome)
        q.append([int(stringa),nome])
    p.append(q)
print(c)

Porte de la Chapelle
Marx Dormoy
Marcadet - Poissonniers
Jules Joffrin
Lamarck - Caulaincourt
Abbesses
Pigalle
Saint-Georges
Notre-Dame-de-Lorette
Trinité - d'Estiennes d'Orves
Saint-Lazare
Madeleine
Concorde
Assemblée nationale
Solferino
Rue du Bac
Sèvres - Babylone
Rennes
Notre-Dame-des-Champs
Montparnasse - Bienvenüe
Falguière
Pasteur
Volontaires
Vaugirard
Convention
Porte de Versailles
Corentin Celton
Mairie d'Issy
Simplon
Marcadet - Poissonniers
Château Rouge
Barbès-Rochechouart
Gare du Nord
Gare de l'Est
Château d'eau
Strasbourg - Saint-Denis
Réaumur - Sébastopol
Etienne Marcel
Les Halles
Châtelet
Cité
Saint-Michel
Odéon
Saint Germain-des-Prés
Saint-Placide
Montparnasse Bienvenüe
Vavin
Raspail
Denfert-Rochereau
Mouton-Duvernet
Alésia
Porte d'Orléans
Mairie de Montrouge
Victor Hugo
Charles de Gaulle Etoile
Ternes
Courcelles
Monceau
Villiers
Rome
Place de Clichy
Blanche
Pigalle
Anvers
Barbès Rochechouart
La Chapelle
Stalingrad
Jaurès
Colonel Fabien
Belleville
Couronnes
Ménilmontant

In [16]:
#### questo script richiede la preventiva esecuzione del precedente
import pymysql
import time
conn = pymysql.connect(host='localhost', user='root', password='root', database='paris_metro')
cur = conn.cursor()
query = "SELECT CODlinea,CODstazione FROM linee_stazioni"
nrows = cur.execute(query)
r = []
for i in range(nrows):
        string = cur.fetchone()
        r.append(string)
print(r)
query = "UPDATE linee_stazioni SET posizione1 = -1, posizione2 = -1 WHERE 1"
nrows = cur.execute(query)
c = 0       
for i in range(len(p)):
    n = len(p[i])
    for j in range(len(p[i])):
        for k in range(len(r)):
            if (r[k][1] == p[i][j][0] and r[k][0] == i+1):
                c = c + 1
                query = "UPDATE linee_stazioni "
                query = query + "SET posizione1 = " + str(j + 1) + "," 
                query = query + "posizione2 = " + str(n - j)
                query = query + " WHERE (CODlinea = " + str(r[k][0])
                query = query + " AND CODstazione = " + str(r[k][1]) +");"
                nrows = cur.execute(query) 
                print(str(c) + " " + str(nrows) + " " + query)
conn.commit()

[(1, 1), (1, 2), (1, 10), (1, 67), (1, 68), (1, 70), (1, 96), (1, 101), (1, 126), (1, 136), (1, 153), (1, 154), (1, 155), (1, 169), (1, 171), (1, 181), (1, 185), (1, 186), (1, 195), (1, 203), (1, 225), (1, 232), (1, 250), (1, 260), (1, 268), (1, 271), (1, 283), (1, 286), (1, 294), (1, 299), (1, 306), (2, 3), (2, 14), (2, 16), (2, 17), (2, 54), (2, 57), (2, 58), (2, 63), (2, 82), (2, 92), (2, 107), (2, 109), (2, 144), (2, 159), (2, 169), (2, 181), (2, 182), (2, 188), (2, 224), (2, 238), (2, 248), (2, 249), (2, 269), (2, 274), (2, 278), (2, 280), (2, 285), (2, 288), (2, 300), (3, 4), (3, 7), (3, 13), (3, 17), (3, 21), (3, 27), (3, 51), (3, 65), (3, 73), (3, 74), (3, 123), (3, 129), (3, 173), (3, 179), (3, 183), (3, 197), (3, 200), (3, 203), (3, 205), (3, 217), (3, 257), (3, 287), (3, 292), (3, 301), (3, 305), (4, 5), (4, 25), (4, 32), (4, 41), (4, 53), (4, 60), (4, 79), (4, 94), (4, 100), (4, 114), (4, 116), (4, 119), (4, 122), (4, 134), (4, 158), (4, 168), (4, 170), (4, 175), (4, 176), 

185 1 UPDATE linee_stazioni SET posizione1 = 12,posizione2 = 8 WHERE (CODlinea = 9 AND CODstazione = 283);
186 1 UPDATE linee_stazioni SET posizione1 = 13,posizione2 = 7 WHERE (CODlinea = 9 AND CODstazione = 152);
187 1 UPDATE linee_stazioni SET posizione1 = 14,posizione2 = 6 WHERE (CODlinea = 9 AND CODstazione = 188);
188 1 UPDATE linee_stazioni SET posizione1 = 15,posizione2 = 5 WHERE (CODlinea = 9 AND CODstazione = 64);
189 1 UPDATE linee_stazioni SET posizione1 = 16,posizione2 = 4 WHERE (CODlinea = 9 AND CODstazione = 172);
190 1 UPDATE linee_stazioni SET posizione1 = 17,posizione2 = 3 WHERE (CODlinea = 9 AND CODstazione = 45);
191 1 UPDATE linee_stazioni SET posizione1 = 18,posizione2 = 2 WHERE (CODlinea = 9 AND CODstazione = 127);
192 1 UPDATE linee_stazioni SET posizione1 = 19,posizione2 = 1 WHERE (CODlinea = 9 AND CODstazione = 106);
193 1 UPDATE linee_stazioni SET posizione1 = 1,posizione2 = 37 WHERE (CODlinea = 10 AND CODstazione = 150);
194 1 UPDATE linee_stazioni SET posizi

In [17]:
### da aggiungere la direzione ed il numero di stazioni da attraversare nella funzione “percorso”
### attualmente i dati nelle colonne “posizione1” e “posizione2” sono incompleti a causa della
### imprecisa denominazione delle stazioni negli elenchi usati 

def direzione(partenza,arrivo,linea):
    import pymysql
    conn = pymysql.connect(host='localhost', user='root', password='root', database='paris_metro')
    cur = conn.cursor()
    query = "SELECT posizione1 FROM linee, linee_stazioni, stazioni "
    query = query + "WHERE IDlinea = CODlinea AND IDstazione = CODstazione "
    query = query + "AND linee.nome = \'" + linea + "\' AND stazioni.nome = \'" + partenza + "\';"
    nrows = cur.execute(query)
    result = cur.fetchone()
    pos_partenza = int(str(result)[1:-2])
    query = "SELECT posizione1 FROM linee, linee_stazioni, stazioni "
    query = query + "WHERE IDlinea = CODlinea AND IDstazione = CODstazione "
    query = query + "AND linee.nome = \'" + linea + "\' AND stazioni.nome = \'" + arrivo + "\';"
    nrows = cur.execute(query)
    result = cur.fetchone()
    pos_arrivo = int(str(result)[1:-2])
    diff = pos_arrivo - pos_partenza
    if (diff > 0):
        a = '2'
    else:
        a = '1'
    query = "SELECT stazioni.nome FROM linee, linee_stazioni, stazioni "
    query = query + "WHERE IDlinea = CODlinea AND IDstazione = CODstazione "
    query = query + "AND posizione" + a + " = 1 AND linee.nome = \'" + linea + "\';"
    nrows = cur.execute(query)
    result = cur.fetchone()
    print(result)
    r = [str(result)[2:-3], abs(diff)]
    return r

In [18]:
### la funzione "direzione" indica la stazione terminale della linea indicata (“3”) nella direzione corretta
### ed il numero di fermate (compresa quella di arrivo)
### non è sempre corretta perché il database non è completo (mancano le posizioni di circa un terzo delle stazioni)

direzione("République","Opéra","3")

('Anatole France',)


['Anatole France', 7]

In [19]:
direzione("Ourcq","Bastille","5")

("Place d'Italie",)


["Place d'Italie", 12]